# Text Mining

## Arbeiten mit Jupyter Notebooks


[Jupyter Notebooks](https://jupyter.org/) sind eine browserbasierte Oberfläche um mit Skriptsprachen, wie [R](https://www.r-project.org/), zu interagieren. Notebooks bestehen aus *Zellen*, die zwei wichtigsten Arten von Zellen sind:
+ Markdown Zellen für formatieren Text und
+ Code Zellen für Quelltext der ausgeführt werden soll.


Um Code in einer Zelle ausführbar zu machen können Sie beispielsweise folgenden R-Quelltext in das untere Textfeld kopieren   

```R
str_variable_name <- "Hallo Jupyter"
print(str_variable_name)
```

Um den Code in der Zelle auszuführen drücken Sie `Shift-Enter`.

## Vergleichende Analyse von Twitter Profilen

Twitter spielt eine zunehmend wichtige Rolle in der politischen Kommunikation. In diesem Notebook wird der Vergleich von der Aktivität von Bernd Riexinger (*Die Linke*) und Alice Weidel (*AFD*) besprochen. Sowohl [Bernd Riexinger](https://twitter.com/b_riexinger) als auch [Alice Weidel](https://twitter.com/Alice_Weidel) haben öffentlich zugängliche Profile auf Twitter. Im Notebook [get_tweets.ipynb](./get_tweets.ipynb) werden diese aus Twitter geladen. Das Notebook baut auf dem R-Paket [tidytext](https://cran.r-project.org/web/packages/tidytext/index.html) auf und folgt dem Anwendungsbeispiel für den [Vergleich](https://www.tidytextmining.com/twitter.html) von Twitter Archiven.

### Import von Daten

Im ersten Schritt importieren wir benötigte R-Pakete. Im Anschluss laden wir die Daten in die Analyseumgebung und werfen einen Blick auf die Verteilung der Häufigkeiten von Tweets über die Zeit.

In [ ]:
library(lubridate)
library(ggplot2)
library(dplyr)
library(readr)
library(tidyr)
library(scales)
library(tidytext)
library(stringr)

In [ ]:
b_riexinger.csv <- read_csv("data/b_riexinger.csv")
alice_weidel.csv <- read_csv("data/alice_weidel.csv")

In [ ]:
tweets <- bind_rows(b_riexinger.csv %>% mutate(collection = "b_riexinger", realname = "Bernd Riexinger"),
                    alice_weidel.csv %>% mutate(collection = "Alice_Weidel", realname = "Alice Weidel")) 

### Was wir importiert haben und der [Pipe](https://uc-r.github.io/pipe) `%>%` Operator

In [ ]:
tweets %>% head()

In [ ]:
tweets %>% select("date", "realname", "text") %>%  head()

In [ ]:
head(tweets[, c("date", "realname", "text")], n = 5)

Wie viel haben wir importiert?

In [ ]:
tweets %>% nrow()

Anzeigen von Dokumentation

In [ ]:
?nrow

In [ ]:
tweets %>% filter(username == "b_riexinger") %>% nrow()

In [ ]:
tweets %>% select("realname") %>% table()

### Darstellung der Verteilung über die Zeit

In [ ]:
tweets %>% ggplot(aes(x = date, fill = collection)) +
  geom_histogram(position = "identity", bins = 20, show.legend = FALSE) +
  facet_wrap(~realname, ncol = 1) + 
  labs(y = NULL, x = "Datum")

### Frequenzanalyse

![alt text](data/vector_space.png "Bag of Words / Vector Space Representation")


[Stoppwörter](https://de.wikipedia.org/wiki/Stoppwort) sind Wörter die häufig auftreten und wenig Relevanz für die Erfassung des Inhalts besitzen.

In [ ]:
stopwords <- get_stopwords(language = "de")$word

In [ ]:
stopwords %>% head()

In [ ]:
tidy_tweets <- tweets %>% 
  filter(!str_detect(text, "^RT")) %>%
  mutate(text = str_remove_all(text, "&amp;|&lt;|&gt;")) %>%
  unnest_tokens(word, text, token = "tweets") %>%
  filter(!word %in% stopwords,
         !word %in% str_remove_all(stopwords, "'"),
         str_detect(word, "[a-z]"),
         nchar(word) > 2,
        )

tidy_tweets %>% head()

Um ausschließlich *#HashTags* zu extrahieren kann alternativ folgender Aufruf
```R
str_detect(word, "#[a-z]")
```
von `str_detect` verwendet werden.

### Von einzelnen Wörtern zu Häufigkeiten

In [ ]:
frequency <- tidy_tweets %>% 
  group_by(collection) %>% 
  count(word, sort = TRUE) %>% 
  left_join(tidy_tweets %>% 
  group_by(collection) %>% 
  summarise(total = n())) %>%
  mutate(freq = n/total) %>% 
  filter(n > 5)

In [ ]:
frequency %>% filter(collection == "Alice_Weidel") %>% head(n=10)

Analog können die `n` häufigsten Wörter für Bernd Riexinger (b_riexinger) ausgegeben werden.

Eine alternative form für die Worthäufigkeiten kann wie folgt berechnet werden.

In [ ]:
frequency2 <- frequency %>% 
  select(collection, word, freq) %>% 
  spread(collection, freq) %>% 
  arrange(Alice_Weidel, b_riexinger) %>% 
  na.omit()

In [ ]:
frequency2 %>% head(n=10)

Die Daten sind nun in einer geeigneten Form für die grafische Darstellung. 

In [ ]:
frequency2 %>% ggplot(aes(Alice_Weidel, b_riexinger)) +
  geom_jitter(alpha = 0.1, size = 2.5, width = 0.25, height = 0.25) +
  geom_text(aes(label = word), check_overlap = TRUE, vjust = 1.5) +
  scale_x_log10(labels = percent_format()) +
  scale_y_log10(labels = percent_format()) +
  geom_abline(color = "blue") + 
  labs(y = "Bernd Riexinger", x = "Alice Weidel")

In der Grafik sehen wir die relative Häufigkeit der Wörter welche von beiden Politikern genutzt wurden.

### Die Wortwahl der Akteure

\begin{equation}
\textrm{Odds-Ratio} = \frac{\frac{n+1_{Weidel}}{total+1_{Weidel}}}{\frac{n+1_{Riexinger}}{total+1_{Riexinger}}},\;\;\;\; \textrm{mit} \;\; \textrm{Log-Odds-Ratio} = log(\textrm{Odds-Ratio})
\end{equation}

**Interpretation:** Einn Log-Odds-Ratio (siehe Chancenverhältnis [Wikipedia](https://de.wikipedia.org/wiki/Chancenverh%C3%A4ltnis)) von:
+ 1 bedeutet, dass es keinen Unterschied hinsichtlich der Verwendungswahrscheinlichkeit eines Wortes gibt,
+ \>1 bedeutet, dass die Wahrscheinlichkeit der Verwendung eines Wortes durch Weidel größer sind,
+ <1 bedeutet, dass die Wahrscheinlichkeit der Verwendung durch Weidel kleiner (durch Riexinger größer) sind.


In [ ]:
word_ratios <- tidy_tweets %>%
  count(word, collection) %>%
  group_by(word) %>%
  filter(sum(n) >= 10) %>%
  ungroup() %>%
  spread(collection, n, fill = 0) %>%
  mutate_if(is.numeric, list(~(. + 1) / (sum(.) + 1))) %>%
  mutate(logratio = log(Alice_Weidel / b_riexinger)) %>%
  arrange(desc(logratio)) %>%
  group_by(logratio < 0) %>%
  top_n(15, abs(logratio)) %>%
  ungroup() %>%
  mutate(word = reorder(word, logratio)) %>% 
  group_by(logratio < 0) %>%
  ungroup() %>%
  mutate(word = reorder(word, logratio))

In [ ]:
word_ratios %>% head()

In [ ]:
word_ratios %>% ggplot(aes(word, logratio, fill = logratio < 0)) +
  geom_col(show.legend = FALSE) +
  coord_flip() +
  ylab("Log-Odds-Ratio (Weidel / Riexinger)") 